In [ ]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

def reshape_4(array):
    return array.reshape(-1, 4)

train_predR = reshape_4(pd.read_csv('VpredR.csv').values)
train_predF = reshape_4(pd.read_csv('VpredF.csv').values)
train_LSTMpredR = reshape_4(pd.read_csv('LSTM_VpredR.csv').values)
train_LSTMpredF = reshape_4(pd.read_csv('LSTM_VpredF.csv').values)

test_predR = reshape_4(pd.read_csv('predR.csv').values)
test_predF = reshape_4(pd.read_csv('predF.csv').values)
test_LSTMpredR = reshape_4(pd.read_csv('LSTM_predR.csv').values)
test_LSTMpredF = reshape_4(pd.read_csv('LSTM_predF.csv').values)


In [ ]:
X_train = np.vstack((np.hstack((train_predR, train_LSTMpredR)),  
                          np.hstack((train_predF, train_LSTMpredF))))  

X_test = np.vstack((np.hstack((test_predR, test_LSTMpredR)),  
                         np.hstack((test_predF, test_LSTMpredF))))  

y_train = np.vstack((np.ones((train_predR.shape[0], 1)),  
                          np.zeros((train_predF.shape[0], 1))))  

y_test = np.vstack((np.ones((test_predR.shape[0], 1)),  
                         np.zeros((test_predF.shape[0], 1))))  

In [ ]:
svm_model = SVC(probability=True)
svm_model.fit(X_train, y_train.ravel())
svm_acc = svm_model.score(X_test, y_test)

print(f"Точність SVM: {svm_acc * 100:.2f}%")

In [ ]:
rf_model = RandomForestClassifier(n_estimators=10, max_depth=8, random_state=42)
rf_model.fit(X_train, y_train.ravel())
rf_acc = rf_model.score(X_test, y_test)

print(f"Точність Random Forest: {rf_acc * 100:.2f}%")

In [ ]:
mlp_model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(32, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    
    Dense(64, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),

    Dense(64, activation='relu'),
    BatchNormalization(),
    
    Dense(1, activation='sigmoid')
])
optimizer = Adam(learning_rate=0.0005)
mlp_model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

mlp_model.fit(X_train, y_train, 
              epochs=10, batch_size=512, 
              #validation_data=(X_test, y_test),
              verbose=1)


mlp_accuracy = mlp_model.evaluate(X_test, y_test, verbose=0)[1]

print(f"Точність Покращеної MLP: {mlp_accuracy * 100:.2f}%")